In [1]:
using KerrGeodesics
using Plots
using GLMakie
using GeometryBasics
using Interpolations

In [6]:
@time EMRI_info = kerr_geo_emri(0.9, 10.0, 0.5, 0.8; initPhases = (0.0, 0.0, 0.0, 0.0))

  0.000211 seconds (482 allocations: 35.719 KiB)


KerrGeoEMRI(
    OrbitalParameters = (a = 0.9, p = 10.0, e = 0.5, x = 0.8),
    ConstantsOfMotion = (E = 0.9641204328952226, Lz = 2.8359152778998453, Q = 4.544408272395823),
    OrbitalType = ["Bound", "Eccentric", "Inclined"],
    Frequencies = (ϒt = 171.0926187383033, ϒr = 2.792721794117058, ϒθ = 3.551489601048812, ϒϕ = 3.7357605214030265),
    Parametrization = "Mino",
    Trajectory = (t = t(λ), r = r(λ), θ = θ(λ), ϕ = ϕ(λ)),
    InitialPhases = (qt0 = 0.0, qr0 = 0.0, qθ0 = 0.0, qϕ0 = 0.0),
)

In [3]:
t, r, θ, ϕ =  EMRI_info.Trajectory
λ_values = range(0, 20, 1000)
t_values = [t(λ) for λ in λ_values]
r_values = [r(λ) for λ in λ_values]
θ_values = [θ(λ) for λ in λ_values]
ϕ_values = [ϕ(λ) for λ in λ_values]

x_values = r_values .* sin.(θ_values) .* cos.(ϕ_values)
y_values = r_values .* sin.(θ_values) .* sin.(ϕ_values)
z_values = r_values .* cos.(θ_values)

a = EMRI_info.OrbitalParameters.a
rp = 1 + sqrt(1 - a^2)

nframes = 900
fps = 30

t_video = range(t_values[1], t_values[end], length=nframes)

itp_x = LinearInterpolation(t_values, x_values)
itp_y = LinearInterpolation(t_values, y_values)
itp_z = LinearInterpolation(t_values, z_values)

fig = Figure(size=(800, 600))
ax = Axis3(fig[1, 1], title="Particle's geodesic trajectory around a Kerr black hole")
sphere_mesh = Sphere(Point3f(0,0,0), rp)
mesh!(ax, sphere_mesh, color=:black)

lines!(ax, x_values, y_values, z_values, color=:gray)

point = Observable(Point3f(itp_x(t_video[1]), itp_y(t_video[1]), itp_z(t_video[1])))
GLMakie.scatter!(ax, point; color=:red, markersize=10)

record(fig, "Trajectory_generic.gif", 1:nframes; framerate=fps) do i
    point[] = Point3f(itp_x(t_video[i]), itp_y(t_video[i]), itp_z(t_video[i]))
end

"Trajectory_generic.gif"